In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Iknur_work/

/content/drive/.shortcut-targets-by-id/1Dr0AezF1S8Tqp3hkotrq7lGVbj2ZF9Fu/Iknur_work


In [ ]:
!pip install snntorch==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install binary_fractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for binary_fractions: filename=binary_fractions-1.1.0-py3-none-any.whl size=47920 sha256=588c3db0c60dae480665f188eef4c35ebaaabc6e3ee619a15d55b8e0ebb88c05
  Stored in directory: /root/.cache/pip/wheels/cb/78/50/5ee188cbaf8195fdaa2d1041c03ae33c9add94ead9131c4713
Successfully built binary_fractions


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from torchvision import datasets, transforms

In [ ]:
def data_loader(dataset = "mnist", in_shape = (28, 28), batch_size = 128):

    dtype = torch.float
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Define a transform
    transform = transforms.Compose([
                transforms.Resize(in_shape),
                transforms.Grayscale(),
                transforms.ToTensor(),
                transforms.Normalize((0,), (1,))])

    if dataset.lower() == 'mnist':
        data_path = './dataset/mnist'
        mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
        mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)
        train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
        test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)
    elif dataset.lower() == 'fashion-mnist':
        data_path = './dataset/fashion-mnist'
        fmnist_train = datasets.FashionMNIST(data_path, train=True, download=True, transform=transform)
        fmnist_test  = datasets.FashionMNIST(data_path, train=False, download=True, transform=transform)
        train_loader = DataLoader(fmnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
        test_loader = DataLoader(fmnist_test, batch_size=batch_size, shuffle=True, drop_last=True)
    elif dataset.lower() == 'cifar10':
        data_path = './dataset/cifar10'
        cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
        cifar10_test  = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)
        train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, drop_last=True)
        test_loader = DataLoader(cifar10_test, batch_size=batch_size, shuffle=True, drop_last=True)

    return train_loader,test_loader

In [ ]:
dataset = "fashion-mnist"
train_loader,test_loader = data_loader(dataset = dataset, in_shape = (28, 28), batch_size = 128)

In [ ]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import snntorch.utils as utils
import torch
import torch.nn as nn

In [ ]:
def print_batch_accuracy(net, batch_size, data, targets, train=False):
    output, _ = net(data)
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer(
    net, batch_size,
    data, targets, epoch,
    counter, iter_counter,
        loss_hist, test_loss_hist, test_data, test_targets):
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print('Average Test Loss: ',np.mean(test_loss_hist))
    print_batch_accuracy(net, batch_size, data, targets, train=True)
    print_batch_accuracy(net, batch_size, test_data, test_targets, train=False)
    print("\n")

In [ ]:
# Define Network
class Net(nn.Module):
    def __init__(self,
                 num_inputs=1,
                 num_hidden_layers=1,
                 num_hidden=1,
                 num_outputs=1,
                 num_steps=10,
                 beta=0.8,
                 vth=1.0,
                 rst='zero',
                 bias=False):
        super().__init__()
        # Num steps
        self.num_steps = num_steps

        # Initialize layers
        # Generate the model architecture to be simulated
        arch = [num_inputs]
        for l in range(num_hidden_layers):
            arch.append(num_hidden)
        arch.append(num_outputs)
        # Number of layers
        self.num_layers = len(arch) - 1

        # Initialize Memory
        self.init_mems = list()

        # Define Layers
        self.layers = nn.ModuleList()
        for i in range(self.num_layers):
            fc  = nn.Linear(arch[i], arch[i+1], bias=bias)
            lif = snn.Leaky(beta=beta,threshold=vth,reset_mechanism=rst)
            mem = lif.init_leaky()
            self.layers.append(fc)
            self.layers.append(lif)
            self.init_mems.append(mem)

    def forward(self, x):
        mems = [self.layers[2*i+1].init_leaky() for i in range(self.num_layers)]

        # Record the final layer
        spk_out = []
        mem_out = []

        for step in range(self.num_steps):
            spk = x[step]
            for i in range(self.num_layers):
                cur = self.layers[2*i](spk)
                spk,mems[i] = self.layers[2*i+1](cur,mems[i])

            spk_out.append(spk)
            mem_out.append(mems[-1])

        return torch.stack(spk_out, dim=0), torch.stack(mem_out, dim=0)

In [ ]:
import torch.nn.utils.prune as prune

In [ ]:
for sp in range(10,51,10):

  print(f"Sparsity {sp}%", "\n\n")
  num_inputs = 28 * 28
  num_hidden = 256
  num_outputs = 10
  num_hidden_layers = 2

  dtype = torch.float
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  # Load the network onto CUDA if available
  net = Net(num_inputs=num_inputs,
            num_hidden_layers=num_hidden_layers,
            num_hidden=num_hidden,
            num_outputs=num_outputs,
            num_steps=100,
            beta=0.95,
            vth=1.0,
            rst="subtract").to(device)
  # Loss and optimizer
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

  # Main training framework starts here
  utils.reset(net)
  loss_hist = []
  test_loss_hist = []
  counter = 0
  weight_scaling = True
  num_epochs = 5
  batch_size =128
  num_steps = 100

  # Pruning Info
  parameters_to_prune = []
  for name,mod in list(net.named_modules())[1:]:

      keys = dict(mod.named_parameters()).keys()

      if dict().keys() == keys:
          continue
      else:
          if 'bias' in keys:
              parameters_to_prune.append((mod,'bias'))

          if 'weight' in keys:
              parameters_to_prune.append((mod,'weight'))

  print("Number of layers to prune: ", len(parameters_to_prune), "\n")

  sparsity_amount = sp
  percentage_amount = (sparsity_amount/100)
  pruning_amount = [percentage_amount  for i in range(len(parameters_to_prune))]
  print(pruning_amount, "\n\n")

  for epoch in range(num_epochs):
      iter_counter = 0
      train_batch = iter(train_loader)

      for data, targets in train_batch:
          data = spikegen.rate(data.view(batch_size,-1),num_steps=num_steps).to(device)
          targets = targets.to(device)
          # forward pass
          net.train()
          spk_rec, mem_rec = net(data)

          # initialize the loss & sum over time
          loss_val = torch.zeros((1), dtype=dtype, device=device)
          for step in range(num_steps):
              loss_val += loss(mem_rec[step], targets)

          # Gradient calculation + weight update
          optimizer.zero_grad()
          loss_val.backward()
          optimizer.step()
          # Update weights
          for model_params in net.parameters():
              if weight_scaling:
                  model_params.data = torch.clamp(model_params.data,min=-0.9375,max=0.9375)

          torch.cuda.empty_cache()

          for j,i in enumerate(parameters_to_prune):
              modu,parm = i
              try:
                  prune.l1_unstructured(modu, name=parm, amount= pruning_amount[j])
                  prune.remove(modu, parm)
              except Exception:
                  pass

          # Store loss history for future plotting
          loss_hist.append(loss_val.item())
          # For inference
          # batch_infer = list()
          # Test set
          with torch.no_grad():
              net.eval()
              test_data, test_targets = next(iter(test_loader))
              test_data = spikegen.rate(test_data.view(batch_size,-1),num_steps=num_steps).to(device)
              #test_data = test_data.to(device)
              test_targets = test_targets.to(device)
              # Test set forward pass
              test_spk, test_mem = net(test_data)
              # Test set loss
              test_loss = torch.zeros((1), dtype=dtype, device=device)
              for step in range(num_steps):
                  test_loss += loss(test_mem[step], test_targets)
              test_loss_hist.append(test_loss.item())
              # Print train/test loss/accuracy
              if counter % 50 == 0:
                  train_printer(
                      net, batch_size,
                      data, targets, epoch,
                      counter, iter_counter,
                      loss_hist, test_loss_hist,
                      test_data, test_targets)
              counter += 1
              iter_counter +=1


  print("\n\n")

  # Evaluation
  pred = []
  target = []
  net.eval()
  for test_data, test_targets in test_loader:
    test_data = spikegen.rate(test_data.view(batch_size,-1),num_steps=num_steps).to(device)
    test_targets = test_targets
    output, _ = net(test_data)
    _, idx    = output.sum(dim=0).max(1)

    pred += idx.detach().cpu().numpy().tolist()
    target += test_targets.numpy().tolist()

  acc = np.mean((np.array(target) == np.array(pred)))
  print(f"Test batch accuracy for {sparsity_amount}% sparsity : {acc*100:.2f}%")
  print("\n\n")

  dict_name = dataset + "_784_256_256_10" + "_with_" + str(sparsity_amount) + "%_Sparsity" + '.dict'
  model_name = dataset + "_784_256_256_10" + "_with_" + str(sparsity_amount) + "%_Sparsity" +'.mdl'
  # Save output
  torch.save(net,model_name)
  torch.save(net.state_dict(),dict_name)

Sparsity 10% 


Number of layers to prune:  3 

[0.1, 0.1, 0.1] 


Epoch 0, Iteration 0
Train Set Loss: 228.28
Test Set Loss: 221.38
Average Test Loss:  221.37973022460938
Train set accuracy for a single minibatch: 51.56%
Test set accuracy for a single minibatch: 33.59%


Epoch 0, Iteration 50
Train Set Loss: 87.40
Test Set Loss: 79.71
Average Test Loss:  118.90832010904948
Train set accuracy for a single minibatch: 67.19%
Test set accuracy for a single minibatch: 62.50%


Epoch 0, Iteration 100
Train Set Loss: 52.64
Test Set Loss: 61.72
Average Test Loss:  100.32002209200718
Train set accuracy for a single minibatch: 78.12%
Test set accuracy for a single minibatch: 79.69%


Epoch 0, Iteration 150
Train Set Loss: 84.32
Test Set Loss: 66.50
Average Test Loss:  90.1168429141013
Train set accuracy for a single minibatch: 71.88%
Test set accuracy for a single minibatch: 75.00%


Epoch 0, Iteration 200
Train Set Loss: 55.19
Test Set Loss: 71.22
Average Test Loss:  84.72512720591986
Train se

# To check sparsity

In [ ]:
w = torch.load("MNIST/mnist_784_256_256_10" + "_with_" + str(20) + "%_Sparsity" + '.dict', map_location=torch.device('cpu'))

In [ ]:
w.keys()

odict_keys(['layers.0.weight', 'layers.1.threshold', 'layers.1.graded_spikes_factor', 'layers.1.reset_mechanism_val', 'layers.1.beta', 'layers.2.weight', 'layers.3.threshold', 'layers.3.graded_spikes_factor', 'layers.3.reset_mechanism_val', 'layers.3.beta', 'layers.4.weight', 'layers.5.threshold', 'layers.5.graded_spikes_factor', 'layers.5.reset_mechanism_val', 'layers.5.beta'])

In [ ]:
totParams = 0
sparseParams = 0
for ix,i in enumerate([w['layers.0.weight'], w['layers.2.weight'], w['layers.4.weight']]):
  var = (i !=0).sum()
  print("Linear Layer ",ix,": ",var, i.shape, "Params :", i.shape[0] * i.shape[1])
  sparseParams += var.item()
  totParams += i.shape[0] * i.shape[1]

print(f"\nTotal Params : {totParams}")
print(f"Sparse Params : {sparseParams}")

Linear Layer  0 :  tensor(160563) torch.Size([256, 784]) Params : 200704
Linear Layer  1 :  tensor(52429) torch.Size([256, 256]) Params : 65536
Linear Layer  2 :  tensor(2048) torch.Size([10, 256]) Params : 2560

Total Params : 268800
Sparse Params : 215040


In [ ]:
# Sparsity

print(f"Sparsity : {(1 - sparseParams/totParams)*100:.2f}%")

Sparsity : 20.00%
